## Пятидневный прогноз погоды в москве

#### Цель

*оптимизировать передачу данных о пятидневном прогнозе погоды с https://openweathermap.org в таблицу облачного PostgreSQL сервера и на ее основе создать интеркативный дашборд в PowerBI.*

### Сетап 

##### Импорт необходимых библиотек

In [110]:
import requests
import pandas as pd
import pytz


##### Получение данных https://openweathermap.org с помощью api в Json формате



In [113]:
url = 'https://api.openweathermap.org/data/2.5/forecast?lat=55.751244&lon=37.618423&appid=62aec7a368b56755f6667fe1dd684b5b&units=metric'
key = '62aec7a368b56755f6667fe1dd684b5b'

In [115]:
response = requests.get(url)
data = response.json()
jdata = data['list']
jdata

[{'dt': 1719705600,
  'main': {'temp': 20.29,
   'feels_like': 20.18,
   'temp_min': 20.29,
   'temp_max': 20.61,
   'pressure': 1014,
   'sea_level': 1014,
   'grnd_level': 996,
   'humidity': 69,
   'temp_kf': -0.32},
  'weather': [{'id': 802,
    'main': 'Clouds',
    'description': 'scattered clouds',
    'icon': '03n'}],
  'clouds': {'all': 42},
  'wind': {'speed': 0.64, 'deg': 311, 'gust': 0.65},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2024-06-30 00:00:00'},
 {'dt': 1719716400,
  'main': {'temp': 20.62,
   'feels_like': 20.54,
   'temp_min': 20.62,
   'temp_max': 21.28,
   'pressure': 1014,
   'sea_level': 1014,
   'grnd_level': 996,
   'humidity': 69,
   'temp_kf': -0.66},
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04d'}],
  'clouds': {'all': 52},
  'wind': {'speed': 1.82, 'deg': 258, 'gust': 2.89},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2024-06-30 03:00:00'},
 {'

##### Перевод данных в pandas DF, используя библиотеку на случай отсутсвия поступления некотрых опциональных данных(например rain.3h или snow.3h)

In [122]:
records=[]
for entry in jdata:
    record = {
    'temp': entry.get('main',{}).get('temp'),
    'feels_like': entry.get('main',{}).get('feels_like'),
    'pressure': entry.get('main',{}).get('pressure'),
    'humidity': entry.get('main',{}).get('humidity'),
    'id': entry.get('weather',[{}])[0].get('id'),
    'main': entry.get('weather',[{}])[0].get('main'),
    'desc': entry.get('weather',[{}])[0].get('description'),
    'clouds': entry.get('clouds',{}).get('all'),
    'wind_speed': entry.get('wind',{}).get('speed'),
    'wind_gust': entry.get('wind',{}).get('gust'),
    'visibility': entry.get('visibility',{}),
    'rain_prob': entry.get('pop',{}),
    'rain(mm)': entry.get('rain',{}).get('3h'),
    'snow(mm)': entry.get('snow',{}).get('3h'),
    'date_for_forcast': entry.get('dt_txt',{}) 
    }
    records.append(record)
    
df = pd.DataFrame(records)
df

,temp,feels_like,pressure,humidity,id,main,desc,clouds,wind_speed,wind_gust,visibility,rain_prob,rain(mm),snow(mm),date_for_forcast
0,20.29,20.18,1014,69,802,Clouds,scattered clouds,42,0.64,0.65,10000,0.00,NaN,None,2024-06-30 00:00:00
1,20.62,20.54,1014,69,803,Clouds,broken clouds,52,1.82,2.89,10000,0.00,NaN,None,2024-06-30 03:00:00
2,23.42,23.44,1014,62,803,Clouds,broken clouds,69,2.09,2.65,10000,0.00,NaN,None,2024-06-30 06:00:00
3,28.71,28.87,1014,46,804,Clouds,overcast clouds,100,1.69,1.95,10000,0.00,NaN,None,2024-06-30 09:00:00
4,31.25,30.68,1013,36,803,Clouds,broken clouds,54,1.66,3.36,10000,0.00,NaN,None,2024-06-30 12:00:00
5,26.96,27.50,1013,52,500,Rain,light rain,92,2.04,3.43,10000,0.32,0.19,None,2024-06-30 15:00:00
6,24.64,24.80,1013,63,500,Rain,light rain,63,2.91,6.16,10000,0.27,0.25,None,2024-06-30 18:00:00
7,21.57,21.98,1013,84,500,Rain,light rain,10,1.42,2.26,10000,0.99,0.78,None,2024-06-30 21:00:00
8,20.52,20.90,1013,87,802,Clouds,scattered clouds,40,0.92,0.88,10000,0.73,NaN,None,2024-07-01 00:00:00
9,21.19,21.58,1013,85,803,Clouds,broken clouds,63,1.60,3.07,10000,0.00,NaN,None,2024-07-01 03:00:00


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   temp              40 non-null     float64
 1   feels_like        40 non-null     float64
 2   pressure          40 non-null     int64  
 3   humidity          40 non-null     int64  
 4   id                40 non-null     int64  
 5   main              40 non-null     object 
 6   desc              40 non-null     object 
 7   clouds            40 non-null     int64  
 8   wind_speed        40 non-null     float64
 9   wind_gust         40 non-null     float64
 10  visibility        40 non-null     int64  
 11  rain_prob         40 non-null     float64
 12  rain(mm)          8 non-null      float64
 13  snow(mm)          0 non-null      object 
 14  date_for_forcast  40 non-null     object 
dtypes: float64(6), int64(5), object(4)
memory usage: 4.8+ KB


### Трансформация данных

##### Изменение типа данных



In [127]:
df['date_for_forcast'] = pd.to_datetime(df['date_for_forcast'])
df['main'] = df['main'].astype('string')
df['desc'] = df['desc'].astype('string')
df['snow(mm)'] = df['snow(mm)'].astype('float')
df.dtypes

temp                       float64
feels_like                 float64
pressure                     int64
humidity                     int64
id                           int64
main                string[python]
desc                string[python]
clouds                       int64
wind_speed                 float64
wind_gust                  float64
visibility                   int64
rain_prob                  float64
rain(mm)                   float64
snow(mm)                   float64
date_for_forcast    datetime64[ns]
dtype: object

##### Изменение временного пояса

In [130]:
df['date_for_forcast'] = df['date_for_forcast'].dt.tz_localize('UTC').dt.tz_convert('Europe/Moscow').dt.tz_localize(None)

##### Заполнение пустых ячеек

In [132]:
df['rain(mm)'] = df['rain(mm)'].fillna(0)
df['snow(mm)'] = df['snow(mm)'].fillna(0)
df.head(3)

,temp,feels_like,pressure,humidity,id,main,desc,clouds,wind_speed,wind_gust,visibility,rain_prob,rain(mm),snow(mm),date_for_forcast
0,20.29,20.18,1014,69,802,Clouds,scattered clouds,42,0.64,0.65,10000,0.0,0.0,0.0,2024-06-30 03:00:00
1,20.62,20.54,1014,69,803,Clouds,broken clouds,52,1.82,2.89,10000,0.0,0.0,0.0,2024-06-30 06:00:00
2,23.42,23.44,1014,62,803,Clouds,broken clouds,69,2.09,2.65,10000,0.0,0.0,0.0,2024-06-30 09:00:00


### Передача данных в облачную БД

In [134]:
!pip install psycopg2 

Defaulting to user installation because normal site-packages is not writeable


In [135]:
import psycopg2 as ps

In [138]:
host='85.193.89.81'
port='5432'
database='default_db'
user='gen_user'
password='@Dm1C21W$Acoq1'

In [139]:
try: conn = ps.connect(host=host,port=port,database=database,user=user,password=password)
except ps.OperationalError as er:
 raise er
else: print('Connected')

Connected


##### Создание пустых строк в таблице БД

In [141]:
def create_rows(table,rows):
 cursor = conn.cursor()
 for i in range(rows):
    insert_query = f""" INSERT INTO {table} DEFAULT VALUES """
    cursor.execute(insert_query)
    conn.commit() 


In [142]:
#create_rows("weather_forcast_5day",len(df))

##### Обновление данных в таблице

In [157]:
queryW = ("""UPDATE "weather_forcast_5day" 
          set  visibility = %s,
          rain_prob = %s,	
          date_for_forcast = %s,	
          "temp"	= %s,
          "feels_like" = %s,		
          "pressure" = %s,
          "humidity" = %s,		
          "clouds" = %s,		
          "wind_speed" = %s,	
          "wind_gust"	= %s,					
          "id" = %s,		
          "main" = %s, 		
          "desc" = %s
          WHERE row_num = %s;""")

In [163]:
cursor.execute?

Docstring: execute(query, vars=None) -- Execute query with bound vars.
Type:      builtin_function_or_method

In [152]:
cursor = conn.cursor()
for index, row in df.iterrows():
    cursor.execute(queryW,(row['visibility'],row['rain_prob'],row['date_for_forcast'],row['temp'],row['feels_like'],row['pressure'],row['humidity'],row['clouds'],row['wind_speed'],row['wind_gust'],row['id'],row['main'],row['desc'],index+1))
conn.commit()

##### Проверка обновления данных

In [155]:
    if cursor.rowcount > 0:
        print(f'{cursor.rowcount} rows updated in the "Fweather" table.')
        conn.commit()
    else:
        print('No rows were updated in the "Fweather" table.')


1 rows updated in the "Fweather" table.
